<a href="https://colab.research.google.com/github/samato88/5-Plus/blob/main/CleanGoldRushOutput_5Plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Take a GR output file and extract OCNs, and EAST OCLC API and Hathi Counts, States Retained In, Held In States, Zone, Retained By Symbol, Held By Symbols
and Region (risk)

Takes 10 hours to run full file - need to optimize!!!

TO DO:
*   With WC_DF - could do changes in slices rather than iterrows and it would be faster. Currently I think it takes about 5 min to work through


DONE (I think):
*   Test by library = location of ocn - I think this is done
*   Rework Directories - 5+ rather than generic
*   Itertuples rather than iterrows -  wccounts df,
*   Fix up NYU OCNs - made a secondary file for lookups of MMSID to OCN based on 079s and put two lines of test data in test file "EAST5+.test.csv" has the last item, 5001, as an NYU that lacks an OCN.
Code looks for both no 035 and no ocn in 035 ***THIS NOT WORKING YET***



In [ ]:
#@title check memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#@title code to skip cells - %%skip

from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [ ]:
#@title import google drive
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()  # Unmount existing mount - trying since not reading new files
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title install and use ipdb debugger, install xl packages
!pip install ipdb
!pip install xlsxwriter
!pip install openpyxl

import ipdb


In [ ]:
#@title imports
import os
import sys
if 'google.colab' in sys.modules:
    sys.path.append('/content/drive/MyDrive/Colab Notebooks/Includes/')

import re
import time
from datetime import datetime
from zoneinfo import ZoneInfo
import csv
import pandas as pd
import openpyxl
from MarkSuspectRegEx import title, publisher
from Parse035FieldRegEx import parse_035, mapPascal # should put pascal somewhere more logical, but this works...
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
#@title Comments on input / output fields
# some code written by ChatGPT, May 2023
# USAGE: python3 CleanGoldRushOutput - MUST UPDATE COUNTS, DIR and FILE for what you want

# To DO: might want to add column of Zones Retained In, currently doing those counts in Allocate2023

# Description:
# given a Gold Rush xlsx file, a file of EAST counts(name defined in dir and file variables below), do the following:
# Parse the 035 field and add a new column for OCN
# Check OCN to see if we have or need an EAST count
# Create a new columns for EAST Count, State of holding, State of retentions, Suspect pub or title, Census zone and regionfor risk modeling
# Create a new sheet for rows missing an OCN, remove those rows from Sheet 1
# Create a new sheet for OCNs lacking an EAST count, also save these as a tsv to feed to API
# Create a new tsv files of Sheet 1 - "ForAllFile_" + sheetname.tsv

## See around line 220 for splitting if combined library file - name/ctrl number
#
# Creates 4 new files - dist_modified, ForAllFile and ONCs4EASTAPI, ONCs4HOLDINGSAPI
### Does not take into account # of EAST holdings, e.g. removing higher than 5, that is done in allocation.

  #   0	A Title(245$a)
  # 1 B Title(245$b)
  #   2	C Author
  #   3	D Control Number(001)
  #   4	E Library
  #   5	F Branch
  #   6	G ISBN
  #   7	H ISSN
  #   8	I Publisher Name
  #   9	J Publisher Number
  #   10	K LC Call Number
  #   11	L Dewey Call Number
  #   12	M SuDocs Call Number
  #   13	N Report Number
  #   14	O General Report Number
  #   15	P Edition
  #   16	Q Action Note(583$f)
  #   17	R Action Note(583$h)
  #   18	S Publication Date
  #   19	T Language
  #   20	U MatchKey <-- required to have a matchkey, using for cuml circ I think
  #   21	V Circ. Count <-- only in reports with circ => move this to position 21 if exists
  #   22	W 035 field
  # 23  X Juvenile Lit.
  # 24  Y Gov Docs(086$a)field


# want output to be:
# A	0	Title
# B	1	Author
# C	2	Library
# D	3	Control Field - 001
# E	4	ISBN
# F	5	Publisher Name
# G	6	LC
# H 7 LC Class
# I	8	Edition
# J	9	Publication Date
# K	10	Language
# L	11	MatchKey
# M	12	035 field
# N	13	OCN
# O	14	EAST Counts
# Q 15  WorldCat Holdings
# R	16	From Sheet * NO OCN Output outputs up to Q
# S	17	Circ. Count - nowadays blank...
# T 18  Juvenile Lit.
# U 19  HathiTrust
# V 20  Doc: 086
# W 21  Suspect
# X 22  States Retained In
# Y 23  Held In States
# Z 24  Zone
# AA 25  Retained By Symbol
# AB 26 Held By Symbols
# AC 27 Region


In [ ]:
#@title Define files and environment
# Start the timer
start_time = time.time()

now = datetime.now()
pacific_tz = ZoneInfo('America/Los_Angeles')
now_pacific = now.astimezone(pacific_tz)
print(f"Starting at: {now_pacific.strftime('%Y-%m-%d %H:%M:%S')}")
counter = 0

############# UPDATE THESE FOR FILE YOU ARE RUNNING #########################
test = False
cloud = True
extension = "txt" # are we dealing with an excel input or large csv file that excel can't handle (txt|, xlsx or csv,)
#########

if cloud:
    path = "/content/drive/MyDrive/Colab Notebooks/Includes/"
    inputfiledir = "/content/drive/MyDrive/Colab Notebooks/5+/"
    libmetadata = "/content/drive/MyDrive/Colab Notebooks/Includes/Member_Metadata.xlsx"; # state, % retained

else:
    path = "/Users/samato/Dropbox/EAST/Gold_Rush/Comparisons/Scripts/"
    inputfiledir = "/Users/samato/Dropbox/EAST/Comparisons/PALNI/" ;

if test:  #### - files for testing/development - ####
    feedbacklinecount = 50
    eastcounts = path + "KNOWN-EAST-HOLDINGS.test.csv" ; #
    wccounts = path + "KNOWN-WC-HOLDINGS.test.csv"
    htcounts = path + "KNOWN-HATHI-HOLDINGS.test.csv"; #
    inputfiledir = inputfiledir + "Test-InputFiles/"
    #file = "GR18Plus"
    file = "EAST5+.test"
    #file = "test"

else:
    feedbacklinecount = 50000
    eastcounts = path + "KNOWN-EAST-HOLDINGS.csv" ; #
    wccounts = path + "KNOWN-WC-HOLDINGS.csv"; #
    htcounts = path + "KNOWN-HATHI-HOLDINGS.csv"; #
    inputfiledir = inputfiledir + "InputFiles/"
    file = "EAST5+"

############# UPDATE THESE FOR FILE YOU ARE RUNNING #########################
libmetadatalibrarycolumn = 1 # 1 is all EAST in GR names, 2 is 2023 CA GR report names. NOTE: PASCAL EAST are herein mapped to EAST23 - use value of 2 for pascal
iscombinedfile = False # this used around line 220 - splits control number 001 if is a combined holding file, e.g. PASCAL
combinecirc = False # does the file have circ data and do we want to combine it
titlerow = 0
titlecolumnSubfieldA = 0
titlecolumnSubfieldB = 1
languagecolumn = 19
publishercolumn = 8
OCN035column = 22
matchoncolumn = 20 # should be ocn (3) or matchkey (20), depending on dataset
#########################

nyufile = inputfiledir + "NYU.csv"
input_file = inputfiledir + file + "." + extension
print(input_file)
print(nyufile)
output_file = inputfiledir + file + '_modified.' + extension
output_csv_file = inputfiledir + "ForALLFile_" + file + '.csv'
#ipdb.set_trace()
output_4EASTAPI_file = inputfiledir + file + '_OCNs4EASTAPI.tsv'
output_4HoldingsAPI_file = inputfiledir + file + '_OCNs4HOLDINGSAPI.tsv'

output_noOCN_file = inputfiledir + "ForALLNoOCNFile-" + file + '.csv'
badrows_file = inputfiledir + file + '_badrows.' + extension
noCurrentOCLC_file = inputfiledir + 'NoCurrentOCLC.csv'


In [ ]:
#@title Read in Gold Rush file
# Read the Excel Gold Rush input file and make a pandas dataframe - df

if extension == "xlsx":
    df = pd.read_excel(input_file, sheet_name=0, engine='openpyxl') # uses first sheet
elif extension == "csv":
    df = pd.read_csv(input_file)
else:
    df = pd.read_csv(input_file, sep='|', on_bad_lines='warn') # Issues a warning when a bad line is found but continues processing the file, skipping the problematic line.
    #print(df.columns, "\n\n")

# remove all the PALNI rows, not counting them in this project. .copy avoids SettingWithCopyWarning,
df = df[df['Library'] != "*PALNI"].copy() # chatGPT suggests .copy efficient enough not to worry

df.reset_index(drop=True, inplace=True)

df['MatchKey Count'] = df.groupby('MatchKey')['MatchKey'].transform('count')
total_rows = len(df)

df['Title'] = '' # this will hold merged 245$a 245$b - done in read each line section

# Group by matchkey and count to make new 'MatchKey Count' column
df['MatchKey Count'] = df.groupby('MatchKey')['MatchKey'].transform('count').fillna(0).astype(int)

df['Title'] = '' # this will hold merged 245$a 245$b - done in read each line section

print("Loaded ", total_rows, "from input file: ", input_file)
#print(df.iloc[1])
print(df.columns)
#print(df['MatchKey Count'])
#print(df['035 field'])
#print(df['MatchKey'])



In [ ]:
#@title Read the EAST Libraries Metadata file
# A 0: inst_id,
# B 1: Name in Gold Rush Reports EAST
# C 2: State,
# D 3: Symbol
# E 4: Est Collection Size
# F 5: # in GR
# G 6: Percent Collection
# H 7: Retention type
# I 8: Census Zone
# J 9: Region
# K 10: Allocation File Name
# L 11: URL (blank)
# M 12: FEMA Zone
# N 13: Name in Members xlsx
# O 14: Cohort (1 , 2)
# P 15: OCLC Number Location


libmetadata_dict = {} # map name to metadata
symboltostate_dict = {} # map symbol to state
libmetadata_df = pd.read_excel(libmetadata, sheet_name='Plus5', dtype=str) #
for index, row in libmetadata_df.iterrows():
    #libname: 0:state, 1:symbols, 2:collection size, 3:retentions, 4: %, 5:zone, 6:region, 7:filename, 8:FEMA, 9:Cohort, 10:OCLC Loc
    libmetadata_dict[row.iloc[libmetadatalibrarycolumn]] = [ row['State'],row['Symbol'], row['Est Collection Size'],row['# in Gold Rush (2022)'], \
                                                             row['Percent Collection'], row['Census Zone'], row['Region'], row['Allocation File Name'], \
                                                             row['FEMA Zone'], row['Cohort'], row['OCLC Number Location']]
    # the following format, using index as position, is depricated - updated above to use iloc or column name
    #libmetadata_dict[row[libmetadatalibrarycolumn]] = [ row[2],row[3], row[4],row[5], row[6], row[8], row[9] ]

    symbols = row['Symbol'].split() # split symbols on space
    for sym in symbols:
        symboltostate_dict[sym] = row['State'] # symbol mapped to state

    if pd.isna(libmetadata_dict[row.iloc[libmetadatalibrarycolumn]][10]):
      print(row.iloc[libmetadatalibrarycolumn], " Needs OCN Location in libmetadata sheet")
    #print(row.iloc[libmetadatalibrarycolumn], "=>", libmetadata_dict[row.iloc[libmetadatalibrarycolumn]][10])

In [ ]:
#@title Read in EAST Counts df, load to east_dict
# Read the TSV eastcounts file into a pandas DataFrame, make a hash on ocn of the east count values
# Headers: oclcNumber	currentOCLC	total_retained_count	retained_symbols	status
eastdf = pd.read_csv(eastcounts, dtype=str)
east_dict = {}

#for index, row in eastdf.iterrows():
for row in eastdf.itertuples(index=True):

    # make sure I haven't screwed up holdings vs retained api counts
    if  bool(re.match(r'^\d.*', str(row.retained_symbols))): # check and see if symbols starts with digit, which is error
        #print("Bad row at line", index+2, "in KNOWN-EAST-HOLDINGS - maybe WC counts got in here:", str(row['retained_symbols']))
        print("Bad row at line", row.Index + 2, "in KNOWN-EAST-HOLDINGS - maybe WC counts got in here:", str(row.retained_symbols))

        continue

    if row.retained_symbols == 'retained_symbols': # it's a header row, skip
        continue
    retainedinstate = []
    eastretained = float(row.total_retained_count)   # nan values bif -> could float or df['my_column'] = df['my_column'].fillna(0).astype(int)
    #              the above might bite you later with column east retained being "#.0"
    currentOCN = row.currentOCLC

    libs = str(row.retained_symbols).split(',')

    for sym in libs: # make retained in state and retained by symbol
        sym = sym.strip()
        if sym != 'nan': # there is a retention
            try:
                retainedinstate.append(symboltostate_dict[sym])
            except Exception as e: # the symbol is not an EAST symbol, report and decrement number of retained copies
                print("Symbol not in libmetadata:", sym, "at row ", index+2, libs)
                #print(e)
                eastretained -= 1

    retainedinstate.sort()
    east_dict[row.oclcNumber] = [eastretained, ",".join(retainedinstate), libs, currentOCN] # OCLC Number:  0:Retentions , 1:States, 2:symbols, 3:current OCN
    if (index + 1) % 50000 == 0:
        print(f"Loaded {index + 1} rows into the EAST hash")
        #print(row[0], ":", east_dict[row[0]])
#print(east_dict[row.iloc[0]][0])
#print(east_dict)


In [ ]:
#@title Create df of >5 and has OCN

print("Known EAST count:", len(eastdf))
# Ensure column 3 is numeric, coercing errors to NaN
eastdf.iloc[:, 2] = pd.to_numeric(eastdf.iloc[:, 2], errors='coerce')

# Now count rows where col 2 is blank and col 3 > 5
count = eastdf[
    (eastdf.iloc[:, 1].isna() | (eastdf.iloc[:, 1] == '')) &
    (eastdf.iloc[:, 2] > 5)
].shape[0]

print("Rows with EAST Greater Than 5, but no Current OCN - should be 0:", count)


In [ ]:
#@title Read in wccounts df, load to wc_dict
# Read the TSV wccounts file into a pandas DataFrame, make a hash on ocn of the east count values
# columns: oclcNumber	currentOclcNumber	mergedOCNs	US Holdings	Holding_count	holding_symbols	title	status
wcdf = pd.read_csv(wccounts, dtype=str)
wc_dict = {}
for index, row in wcdf.iterrows(): # 0:oclcNumber	1:currentOclcNumber	2:mergedOCNs	3:US Holdings	4:Holding_count	5:holding_symbols	6:title	7:status
    if str(row['Holding_count']) == "success":  #if  bool(re.search(r'[a-zA-z]', str(row[3]))): # make sure it is a digit, if not print warning and continue
        print("Bad row at line", index+2, "in KNOWN-WC-HOLDINGS - maybe EAST counts got in here, cell 4 is '" , row['Holding_count'], "' but should be numeric")
        continue
    elif row['status'] == "failed - invalid OCLC number":
        wc_dict[row['oclcNumber']] = "Invalid OCLC Number"
    elif row['status'] == "failed":
        wc_dict[row['oclcNumber']] = "failed lookup"
    else:
        wc_dict[row['oclcNumber']] = row['Holding_count']

    if (index + 1) % 50000 == 0:
        print(f"Loaded {index + 1} rows into the WC hash")
        #print(row[0], ":", east_dict[row[0]])
#print(wc_dict)

In [ ]:
#@title Read in ht_dict
#htdf = pd.read_csv(htcounts, dtype=str, sep='\t') # not sure why I was doing the df step,(excel?) now just reading from file
ht_dict = {}
with open(htcounts, 'r') as ht:
    reader = csv.reader(ht, delimiter="\t")
    ht_dict = {row[0]: row[1] for row in reader}


In [ ]:
#@title Read in NYU_dict
#htdf = pd.read_csv(nyufile, dtype=str) #  now just reading from file, though df is fast
NYU_dict = {}
with open(nyufile, 'r') as nyu:
    reader = csv.reader(nyu)
    NYU_dict = {row[0]: row[1] for row in reader} # BIBID/001 is key, 079 is value
    #NYU_dict = {row[0].lstrip('0'): row[1] for row in reader}  # Strip leading zeros from key (bibid), value is 079
    # above was problematic as BIBID/001 not always numberic for other schools so didn't want to set as int
print(len(NYU_dict))
print(NYU_dict['001187186'])

In [ ]:
#@title If 'iscombinedfile', split out the library name - dependent on first read of GR file!
# Split the data in the 2nd position and update the 3rd position -- ONLY NEED THIS IF COMBINED FILE, EG ODU-1234567
if iscombinedfile:
    #print(df.iloc[:, 3])
    split_data = df.iloc[:, 3].str.split('-', expand=True) # split column D on dash
    df.iloc[:, 4] = split_data[0] # library name
    df.iloc[:, 3] = split_data[1] # control number
    #print(split_data[0])
    #print(split_data[1])

    #df['Library'] = df['Library'].replace(pascalname_dict) # revive if you want to map PALNI names more?
#print(df.columns)

In [ ]:
#@title Fix up call number / add columns and arrays for east, wc, ht, states, symbols, zone, region, suspect, currentOCN

# If callnumber doesn't start with alpha character, make it blank.
df['LC Call Number'] = df['LC Call Number'].apply(lambda x: x if re.match(r'^[a-zA-Z]', str(x)) else '')

# Make 'LC call number' be the value of the leading alpha numeric characters
df['LC Call Number'] = df['LC Call Number'].replace(r'^([a-zA-Z]+[0-9]+).*$', r'\1', regex=True)

# Make a new column, "LC" that is just the beginning alpha
df['LC'] = df['LC Call Number'].replace(r'^([a-zA-Z]+).*$', r'\1', regex=True)

# if this is the circ sheet, move the circ column to the end so other columns in same order as other output sheets
# this might not need to be done anymore...?

if "Circ. Count" in df.columns:
    df["Circ. Count"] = df.pop("Circ. Count")

df.loc[:, "Suspect"] = "" # create column "Suspect"
df.loc[:, "Held In States"] = "" # create column "Held In States"
df.loc[:, "Held By Symbols"] = "" # create column "Held In States"
df.loc[:, "Retained In States"] = "" # create column "States Retained In"
df.loc[:, "Retained By Symbol"] = "" # create column "Retained By Symbol"
df.loc[:, "Zone"] = "" # create column "Zone"
df.loc[:, "Region"] = "" # create column "Region"
df.loc[:, "Current OCN"] = "" # create current OCN column


# New dataframe which will contain rows without OCN
no_ocn_df = pd.DataFrame(columns=df.columns)

# Convert 035 field to text format, rm =, for things like "=035 ..." which were being interpretted as broken formulas.
# GR might not export '=035' prefix anymore .. so this might no longer be necessary
df['035 field'] = df['035 field'].astype("string")
df.loc[:, '035 field'] = df.loc[:, '035 field'].str.replace(r'=', '') # works with regex

ocnmatch = parse_035()  # defined in Parse035FieldRegEx.py # print(ocnmatch.pattern)
titlepattern = title() # defined in MarkSuspectRegEx.py
publisherpattern = publisher() # defined in MarkSuspectRegEx.py

ocn_values = []
rows_to_remove = []
bad_rows = []
east_counts = [] # array to hold EAST counts
east_states = [] # array to hold states in which OCN retained
east_symbols = []# array to hold symbols by whom OCN retained
wc_counts = []
ht_counts = []
needs_east_count = []
needs_wc_count = []
zone = [] # array to hold census zone North South
region = [] # array to hold census region
suspect = [] # array to hold if title/publisher is on suspect list
current_OCN = [] # array to hold current OCN

cumulative_circ = {} # also do cum circ here as we are iterating the file
cumulative_states = {} # also do states here as we are iterating the file
cumulative_holdings_symbols = {} # dict for held by symbols


In [ ]:
#@title Create Title column from 245$a + 245$b
# Replace NaN values with empty strings in the title columns
df['Title(245$a)'] = df['Title(245$a)'].fillna('')
df['Title(245$b)'] = df['Title(245$b)'].fillna('')
# Concatenate the title subfields and assign to the 'Title' column
df['Title'] = df.apply(lambda row: (row['Title(245$a)'] + ' ' + row['Title(245$b)']).strip(), axis=1)

In [ ]:
#@title Fix any bad NYU lines that lack an 035
# should also maybe here just update all 035s with 079?? NYU_dict
#for i in range(len(df)):
 # ipdb.set_trace()

#mask = (df['035 field'].isna() | (df['035 field'] == '')) & (df['Library'] == 'New York University')
mask = ( df['035 field'].isna() | (df['035 field'] == '') | (df['035 field'] == '<NA>') ) & (df['Library'] == 'New York University')
#print(df[mask])


print("Bad NYU Lines that will try to fix: ", len(df[mask]), " these are no 035, bad 035s handled elsewhere")
for idx, row in df[mask].iterrows():
    key = row['Control Number(001)']
    #ipdb.set_trace()
    if key in NYU_dict:
        df.at[idx, '035 field'] = NYU_dict[key]

In [ ]:
#@title Iterate DF, find OCNs, do EAST etc. Lookups, Create Output Arrays
badrowscount = 0

for i in range(len(df)): # this section works on making new OCLC column, and makes dictionary of cumulative circ and states

    ocns = [] # reset each run
    matchkey = df.loc[i,'MatchKey'] # use matchkey as key for states and cum circ and holding symbols

    if combinecirc: # only do this if we have circ in this file
      if "Circ. Count" in df.columns: # if we have a file with circ, make dict to count total circ per matchkey
          if pd.isna(df.loc[i,'Circ. Count']):  ## NEED TO CHECK HERE THAT NOT BLANK OR UNDEF CIRC
            df.loc[i,'Circ. Count'] = 0 # assign 0 to any blanks - this might be a bad idea? is there a difference between zero and no info?
          if matchkey in cumulative_circ.keys():
              cumulative_circ[matchkey] = cumulative_circ[matchkey] + df.loc[i,'Circ. Count']
          else:
              cumulative_circ[matchkey] = df.loc[i,'Circ. Count']

    try: # check to make sure input file has valid value in 'library' column
      libmetadata_dict[df.loc[i, 'Library']][10]
    except Exception as e: #
      #print(e , " at ", i)
      #print(df.loc[i])
      # these usually are blank rows.... note NYU blank 035s fixed above
      badrowscount = badrowscount+1
      bad_rows.append(i)

      ocn_values.append('')
      east_counts.append('')
      east_states.append('')
      east_symbols.append('')
      wc_counts.append('')
      ht_counts.append('')
      region.append('')
      zone.append('')
      suspect.append('')
      current_OCN.append('')
      continue
      #print(df.loc[i])
      #ipdb.set_trace()

    if libmetadata_dict[df.loc[i, 'Library']][10] == "035 field": # if key error here something wrong in source data from GR
      try: # Split the values in column 20 based on "\s" and loop over the array to find OCN

        temp = df.loc[i, '035 field']
        temp = re.sub(r'\s+\$9', '$9', temp) # Keeping $9ExL with ocns for checking for SCSU -  $a(scbph)b10566740-01pascal_scsu $a00846736 $9ExL
        temp = re.sub(r'LC\)\s+', 'LC)', temp) # first remove spaces that seem legit  $a(OCoLC) 1154664360, then,
        temp = re.sub(r'\s+$', '', temp) # remove trailing whitespace
        ocns = re.split('\s+', temp) # split 035 on whitespace, a little wacky but seems to work
        #ipdb.set_trace()
      except Exception as e: # these are blank 035s usually
        #print("error: ", e)
        #print(df.iloc[i,20])
        #print(ocns)
        ocn_values.append('')
        east_counts.append('')
        east_states.append('')
        east_symbols.append('')
        wc_counts.append('')
        ht_counts.append('')
        region.append('')
        zone.append('')
        suspect.append('')
        current_OCN.append('')

        no_ocn_df[len(no_ocn_df)] = df.iloc[i, :]
        rows_to_remove.append(i)
        continue

    else: # assuming ocn in 001, also assuming not blank
      #ipdb.set_trace()
      ocn = df.loc[i, 'Control Number(001)']
      # Convert ocn to string if it's not already - might be blank?
      ocn = str(ocn)
      #try:
      ocn = re.sub(r'^[a-z]+', '', ocn)  #remove leading letters
      ocn = re.sub(r'\s+$','', ocn) # remove trailing whitespace
      ocn = re.sub(r'^[0]+', '', ocn)  #remove leading zeros
      ocn = "(OCoLC)" + ocn # add a prefix so we know this is ocn in the next section
      #ocn = ocn.str.replace(r'^[a-z]+', '', regex=True) #remove leading letters
      #ocn = ocn.str.replace(r'^[0]+', '', regex=True) #remove leading zeros
      ocns = [ocn] # put the ocn in an ocns array
      # except Exception as e:
      #  print(e, "001 was:", ocn)
      #  print(df.loc[i])

      # might be more effient to do a lot of this by slices - e.g. make oclcNumber column

    for oclcnumb in ocns:  #
        if (len(ocns) < 1):
          ipdb.set_trace() # leaving this in for now as it really shouldn't happen and want to see it if it does
        oclcnumb = re.sub(r'^.*?\$a', "", oclcnumb) # remove anything up to and including first $a (gets the 035 field prefix that Gold Rush puts in)
        oclcnumb = re.sub(r'\\+$', "", oclcnumb) # remove any trailing slashes (weird that there are some...)
        oclcnumb = re.sub(r'\(CStRLIN\)', '', oclcnumb) #  $a(CStRLIN)ocn476563480$9ExL;
        oclcnumb = re.sub(r'\(DLC.*\)', '', oclcnumb) # $a(DLC    )ocn478351245$9ExL;
        oclcnumb = re.sub(r'\(\(OCoLC\)\)', '(OCoLC)', oclcnumb) # Bowdoin had some ((OCoLC))
        #ipdb.set_trace()

        ### somtimes these already start with OCoLC
        if str(df.loc[i, 'Library']).startswith("South Carolina State University") and re.search('\$9ExL', oclcnumb) and not oclcnumb.startswith("(OCoLC)"): # hack for SCSU no prefix but is OCN $9ExL
            oclcnumb = "(OCoLC)" + oclcnumb
        if str(df.loc[i, 'Library']).startswith("Bryn Mawr") and re.search('\$9ExL', oclcnumb) and not oclcnumb.startswith("(OCoLC)"): # hack for Brwn Mawr $a(TriCo)b16376626-01tri_inst $a9557018 $9ExL
            oclcnumb = "(OCoLC)" + oclcnumb
        if str(df.loc[i, 'Library']).startswith("Adelphi") and re.search('\$9ExL', oclcnumb) and not oclcnumb.startswith("(OCoLC)"): # hack for Adelphi $a(NGcA)b13378910-01adelphi_inst $a20825739 $9ExL
            oclcnumb = "(OCoLC)" + oclcnumb
        if str(df.loc[i, 'Library']).startswith("Swarthmore") and re.search('\$9ExL', oclcnumb) and not oclcnumb.startswith("(OCoLC)"): # hack for Adelphi $a(NGcA)b13378910-01adelphi_inst $a20825739 $9ExL
            oclcnumb = "(OCoLC)" + oclcnumb

        if '$' in oclcnumb: # if has $ -e.g. a $z or $9 - remove all of that
            oclcnumb = oclcnumb.split('$')[0]

        # NYU had some OCNs in 079 |a(OCoLC)244475171 get it and make it the oclc number. blank 035s already dealt with above
        if not ocnmatch.search(oclcnumb) and df.loc[i, 'Library'] == "New York University":
          key = df.loc[i, 'Control Number(001)']
          if key in NYU_dict:
            if test: #002114878,(OCoLC)28114500
              print("bibid", key, "OCN:", NYU_dict[key], "at", i, "ocn was", oclcnumb, "this should print just one bibid" ) # in test data this should only print once, no 035 should have been fixed by here.
              assert(key == "2114878")
            df.loc[i, '035 field'] = NYU_dict[key] # don't really need to do this - too late by here.
            oclcnumb = NYU_dict[key]

        if ocnmatch.search(oclcnumb) or \
          (str(df.loc[i, 'Library']).startswith("Lafayette") and oclcnumb.isnumeric()) or \
          (str(df.loc[i, 'Library']).startswith("ODU") and oclcnumb.isnumeric()) or \
          (str(df.loc[i, 'Library']).startswith("Bryn Mawr") and oclcnumb.isnumeric()):   # # 'or' hack ODU-001 ^ODU && no prefix is an ocn, Lafayette, there were ~350 Bryn Mawr of these - might be problematic

            oclcnumb = re.sub(ocnmatch, "", oclcnumb) # remove the matched prefix
            oclcnumb = re.sub("^0+", "", oclcnumb) # remove leading zeros
            #print(oclcnumb)
            if oclcnumb.isnumeric() or oclcnumb.isdigit(): # oclcnumb might be an int or a sting at this point...might not need this or
                ocn_values.append(oclcnumb)
                libraryname = df.loc[i, 'Library']   # doing state count here so only counts states in records that have an OCN
                #ipdb.set_trace()
                #libmetadatalibrarycolumn = [ row['State'],row['Symbol'], row['Est Collection Size'],row['# in Gold Rush (2022)'], row['Percent Collection'], row['Census Zone'], row['Region'] ]
                zone.append(libmetadata_dict[libraryname][5]) # #libname: 0:state, 1:symbols, 2:collection size, 3:retentions, 4: percent, 5:census zone, 6:region
                region.append(libmetadata_dict[libraryname][6]) # #libname: 0:state, 1:symbols, 2:collection size, 3:retentions, 4: percent, 5:census zone, 6:region

                if matchkey in cumulative_states.keys():                 # loading up the states dictionary, appending if matchkey already seen
                    cumulative_states[matchkey] = cumulative_states[matchkey] + ', ' + str(libmetadata_dict[libraryname][0])
                else:
                    cumulative_states[matchkey] = libmetadata_dict[libraryname][0]

                holdings_symbol = libmetadata_dict[libraryname][1].split() # want just the first symbol if there are multi
                if matchkey in cumulative_holdings_symbols.keys():                 # loading up the holdings symbols dictionary, appending if matchkey already seen
                    cumulative_holdings_symbols[matchkey] = cumulative_holdings_symbols[matchkey] + ', ' + str(holdings_symbol[0])
                else:
                    cumulative_holdings_symbols[matchkey] = holdings_symbol[0]

                if oclcnumb in east_dict: # do we have an east count for this, and retained states
                    # east_dict => OCLC Number:  0:Retentions , 1:States, 2:symbols, 3:current OCN
                    east_counts.append(east_dict[oclcnumb][0])
                    east_states.append(east_dict[oclcnumb][1])
                    east_symbols.append(",".join(east_dict[oclcnumb][2]))
                    current_OCN.append(east_dict[oclcnumb][3])

                else:
                    east_counts.append('')
                    east_states.append('')
                    east_symbols.append('')
                    current_OCN.append('')
                    needs_east_count.append(oclcnumb)
                if oclcnumb in wc_dict: # do we have a wc count for this
                    wc_counts.append(wc_dict[oclcnumb])
                else:
                    wc_counts.append('')
                    needs_wc_count.append(oclcnumb)
                if oclcnumb in ht_dict: # do we have a ht rights for this
                    ht_counts.append(ht_dict[oclcnumb])
                else:
                    ht_counts.append('')

                # # Finding and marking suspect titles (doing here since known ocn)

                title = str(df.loc[i, "Title"])
                language  = str(df.iloc[i,languagecolumn])
                publisher = str(df.iloc[i,publishercolumn]) # weirdly there were some int publishers...
                #print(titlepattern.search(title))
                if title is not None and language == "English" and titlepattern.search(title):
                    suspect.append("title")
                elif publisher is not None and publisherpattern.search(publisher):
                    suspect.append("publisher")
                else:
                    suspect.append('') # empty if doesn't match title or publisher (this cleans up previous runs and regex fixes)
                break
    else: # else we went through all the v in value w/o finding an ocn # This else runs if the for loop didn't break
      ocn_values.append('')
      east_counts.append('')
      east_states.append('')
      east_symbols.append('')
      wc_counts.append('')
      ht_counts.append('')
      region.append('')
      zone.append('')
      suspect.append('')
      current_OCN.append('')

      #no_ocn_df = no_ocn_df.append(df.iloc[i, :], ignore_index=True)  # Add row to no OCN dataframe
      no_ocn_df[len(no_ocn_df)] = df.iloc[i, :]
      rows_to_remove.append(i)
      #ipdb.set_trace()

print("Bad Rows: ", badrowscount)


In [ ]:
#@title Create Output frames and files

# Add the OCN and EAST values as a new columns
print("length ocn_values    : ", len(ocn_values))
print("length df            : ", len(df))
print("length wc            : ", len(wc_counts))
print("length ht            : ", len(ht_counts))
print("length east_symbols  : ", len(east_symbols))
print("length region        : ", len(region))
print("length zone          : ", len(zone))
print("length suspect       : ", len(suspect))
print("lenght current ocn.  : ", len(current_OCN))

df['OCN'] = ocn_values
df['EAST Counts'] = east_counts
df['States Retained In'] = east_states
df['Retained By Symbol'] = east_symbols
df['WorldCat Holdings'] = wc_counts
df['HathiTrust'] = ht_counts
df['Region'] = region
df['Zone'] = zone
df['Suspect'] = suspect
df['Current OCN'] = current_OCN

# Create a new DataFrame with the dropped rows
dropped_df = df.loc[rows_to_remove]
dropped_df['From Sheet'] = "NO_OCN"
# Drop the rows that need to be removed from the original DataFrame - no ocns
df = df.drop(rows_to_remove)

# Create a new DataFrame with the bad data rows
bad_df = df.loc[bad_rows] # this bad_df gets written out at the end
# Drop the rows that need to be removed from the original DataFrame - bad rows
df = df.drop(bad_rows)

# add from sheet to df
match file:
    case "Unique_0000_1985":
        df['From Sheet'] = 'Unique'
    case "Unique_1986_1999":
        df['From Sheet'] = 'Unique'
    case "Unique_2000_2010":
        df['From Sheet'] = 'Unique'
    case "Overlap_3_4":
        df['From Sheet'] = 'Topping Up'
    case "HighCircLowEAST_MoreThan10":
        df['From Sheet'] = 'High Circ Low EAST'
    case "Pre1900_BOOK_2_5":
        df['From Sheet'] = 'Pre1901'
    case _:
         df['From Sheet'] = file

# Remove specified columns
#columns_to_remove = [4, 6, 8, 10, 11, 12, 13, 15, 16]
#df = df.drop(df.columns[columns_to_remove], axis=1)
#ipdb.set_trace()
df.drop(columns=['Branch', 'ISSN', 'Publisher Number', 'Dewey Call Number', 'SuDocs Call Number', 'Report Number', 'General Report Number', 'Action Note(583$f)', 'Action Note(583$h)'], inplace=True)
# reorder columns from stack overflow : In the end, the point is: df = df[ list with newly arranged column names ] e.g. df = df[['mean', 'average', '1', '2', '3']]
#ipdb.set_trace()
df.rename(columns = {'Gov Docs(086$a)':'Doc: 086', 'LC':'LC Class', 'LC Call Number':'LC'}, inplace = True)
dropped_df.rename(columns = {'Gov Docs(086$a)':'Doc: 086', 'LC':'LC Class', 'LC Call Number':'LC'}, inplace = True)

#df = df[['Title', 'Author', 'Library', 'Control Number(001)', 'ISBN', 'Publisher Name', 'LC', 'LC Class','Edition', 'Publication Date', 'Language', 'MatchKey', '035 field', 'OCN', 'EAST Counts', 'WorldCat Holdings', 'From Sheet', 'Circ. Count', 'Juvenile Lit.', 'HathiTrust', 'Doc: 086', 'Suspect', 'States Retained In','Held In States', 'Zone', 'Retained By Symbol', 'Held By Symbols', 'Region']]
df = df[['Title', 'Author', 'Library', 'Control Number(001)', 'ISBN', 'Publisher Name', 'LC', 'LC Class','Edition', 'Publication Date', 'Language', 'MatchKey', '035 field', 'OCN', 'Current OCN', 'EAST Counts', 'MatchKey Count', 'WorldCat Holdings', 'Juvenile Lit.', 'HathiTrust', 'Doc: 086', 'Suspect', 'States Retained In','Held In States', 'Zone', 'Retained By Symbol', 'Held By Symbols', 'Region']]
                                                                                                                                                                                         #print(df['MatchKey Count'])
dropped_df = dropped_df[['Title', 'Author', 'Library', 'Control Number(001)', 'ISBN', 'Publisher Name', 'LC', 'LC Class','Edition', 'Publication Date', 'Language', 'MatchKey', '035 field', 'OCN', 'Current OCN', 'EAST Counts', 'WorldCat Holdings', 'From Sheet']]

# using set() to remove duplicates from list of OCNs 4 API
needs_east_count = list(set(needs_east_count))
needs_wc_count = list(set(needs_wc_count))

if combinecirc: # if this file has circ and we combined it
  if "Circ. Count" in df.columns: ## fix up circ counts to be cumulative based on matchkey built in cum circ dict
    #df["Circ. Count"] = df["MatchKey"].map(cumulative_circ) #
    df["Circ. Count"] = df.iloc[:, matchoncolumn].map(cumulative_circ) # note using matchoncolumn varialble not 'MatchKey' column

#ipdb.set_trace()
#df["Held In States"] = df["MatchKey"].map(cumulative_states) # add states
#df["Held By Symbols"] = df["MatchKey"].map(cumulative_holdings_symbols) # add held by symbols

df["Held In States"] = df.iloc[:, matchoncolumn].map(cumulative_states) # add states
df["Held By Symbols"] = df.iloc[:, matchoncolumn].map(cumulative_holdings_symbols) # add held by symbols


# Save the modified DataFrame to a new Excel file
df['Control Number(001)'] = df['Control Number(001)'].astype(str) # make sure 001 is string - prevents mmsids from being munged

if extension == "xlsx":
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer: # create a excel writer object
        df.to_excel(writer, sheet_name="Sheet1", index=False, header=True, freeze_panes=(1,0)) #this fails to create valid file if cell started with "=" and is not an equation
        dropped_df.to_excel(writer, sheet_name="no OCN", index=False)

        # create new df for ocns w/o an EAST count that will write to new sheet
        pd.DataFrame({"oclcNumber": needs_east_count}).to_excel(writer, sheet_name="OCLC4EASTAPI", index=False)
        pd.DataFrame({"oclcNumber": needs_wc_count}).to_excel(writer, sheet_name="OCLC4HoldingsAPI", index=False)

        # Access the XlsxWriter workbook and worksheet objects
        workbook  = writer.book
        worksheet = writer.sheets["Sheet1"]

        #text_format = workbook.add_format({'num_format': '0'}) # 0 works better than @, @ gives long numbers in scientific notation

        # Set column - first column, last column, width, format, options (optional)
        #worksheet.set_column('A:D', 20, text_format)  # Column A:D width
        worksheet.set_column('A:D', 20)  # Column A:D width
        worksheet.set_column('E:E', 15)  # Column E width
        worksheet.set_column('F:F', 20)  # Column F width
        worksheet.set_column('G:G', 5)  # Column G width
        worksheet.set_column('J:J', 12)  # Column J width
        worksheet.set_column('V:W', 20)  # Column V thru W width


        # Define a format for wrapped text
        header_format = workbook.add_format({'text_wrap': True, 'bold': True, 'bg_color': 'silver'}) # silver is #C0C0C0
        # Apply the format to the header row
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
else: # else output as csv
    df.to_csv(output_file, encoding='utf-8-sig', index=False)     # this will be the same as ForALL file, so a bit redundant

pd.DataFrame({"oclcNumber": needs_east_count}).to_csv(output_4EASTAPI_file, sep='\t', index=False)
# if test:
#   print(needs_east_count)
pd.DataFrame({"oclcNumber": needs_wc_count}).to_csv(output_4HoldingsAPI_file, sep='\t', index=False)

if combinecirc:
  df['Circ. Count'] = df['Circ. Count'].astype(int) # circ is float64 to account for nan, changed to int for export to avoid 0.0
  #print(df.dtypes)

#df.to_csv(output_tsv_file, sep='\t', encoding='utf-8', index=False)     # this had encoding issues, and utf-8-sig messed up tsv, now using csv
df.to_csv(output_csv_file, encoding='utf-8-sig', index=False)     # Save 'Sheet1' as a CSV file for final merged sheet
dropped_df.to_csv(output_noOCN_file, encoding='utf-8-sig', index=False) # Save the no OCN df/sheet to a csv for final merged sheet
bad_df.to_csv(badrows_file, encoding='utf-8-sig', index=False)

# End the timer and report how long it took
end_time = time.time()
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)
print("----Summary---")
print(total_rows, " rows were in input file")
print(len(rows_to_remove), " rows lacked an OCN")
print(len(needs_east_count), " ONCs Need an EAST count API Lookup")
print(len(needs_wc_count), " ONCs Need a WC holdings Count API Lookup")
print("Bad Rows: ", badrowscount)
now = datetime.now()
pacific_tz = ZoneInfo('America/Los_Angeles')
now_pacific = now.astimezone(pacific_tz)
print(f"\nEnding at: {now_pacific.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Script took {minutes} minutes and {seconds} seconds to run.")

